# Philo Data Science Take-home Excercise

Hello and thanks for taking the time to read through my take-home excercise. I have to admit while I was aware of Wordle I had never played the game before, but this excercise was a lot of fun to think about and work through and I may have to start playing now!

I decided to tackle option 1 of the excercise: 

#### What is the top starting word or words for the game, but the dictionary changes to one of the following (your choice):
#### a. a different language (your choice)
#### b. the complete works of shakespeare ([https://www.gutenberg.org/files/100/100-0.txt](https://www.gutenberg.org/files/100/100-0.txt))

The collected works of William Shakespeare were easily accessible, so I chose that as my dictionary but my approach could also be used for the spanish dictionary option by simply substituting the files.

In [353]:
import pandas as pd
import urllib
import re
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.image as mpimg
from itertools import islice
from tqdm.notebook import tqdm
from collections import defaultdict
from collections import Counter
import ipywidgets as widgets
from ipywidgets import interactive_output, fixed
%matplotlib inline

The first thing to do is read in the dictionary and then make a list of all the five-letter words that appear in the body of the works (i.e., ignore the header and legal notice at the bottom of the file). Then we print that out to get a sense of what we are working with. 

In [354]:
five_letter_words = list()
with open('100-0.txt') as f:
    for line in f.readlines()[28:170290]:
        five_letter_words += re.findall(r'\b([a-zA-Z]{5})\b', line)
    for count, line in enumerate(f):
        pass
    print('Total Lines', count + 1);
print (five_letter_words, len (five_letter_words));

Total Lines 170639
['FIRST', 'HENRY', 'HENRY', 'HENRY', 'FIFTH', 'FIRST', 'HENRY', 'SIXTH', 'HENRY', 'SIXTH', 'THIRD', 'HENRY', 'SIXTH', 'HENRY', 'MERRY', 'WIVES', 'NIGHT', 'DREAM', 'ABOUT', 'THIRD', 'ROMEO', 'SHREW', 'TIMON', 'TITUS', 'NIGHT', 'NOBLE', 'LOVER', 'VENUS', 'might', 'never', 'riper', 'might', 'thine', 'light', 'flame', 'where', 'sweet', 'cruel', 'world', 'fresh', 'gaudy', 'thine', 'churl', 'waste', 'world', 'world', 'grave', 'forty', 'shall', 'field', 'youth', 'proud', 'gazed', 'small', 'worth', 'being', 'asked', 'where', 'Where', 'lusty', 'thine', 'shame', 'child', 'Shall', 'count', 'thine', 'blood', 'glass', 'Whose', 'fresh', 'world', 'where', 'whose', 'glass', 'Calls', 'April', 'prime', 'thine', 'shalt', 'thine', 'image', 'spend', 'gives', 'being', 'frank', 'lends', 'those', 'abuse', 'given', 'great', 'canst', 'alone', 'sweet', 'calls', 'audit', 'canst', 'leave', 'Which', 'lives', 'Those', 'hours', 'frame', 'where', 'every', 'dwell', 'which', 'excel', 'never', 'leads',

Once I scanned the list, I noticed there were multiple versions of the same words and case issues where the same word appeared in mulptiple case formats, so I made every character lowercase and only grabbed the unique words (I preserved the order but it doesn't really matter for this excercise). 3,363 words seemed like a reasonable number even if some of them were different languages or weird old English spellings.

In [355]:
word_list = []
marker = set()

for l in five_letter_words:
    UL = l.lower()
    if UL not in marker:   # test presence
        marker.add(UL)
        word_list.append(UL)   # preserve order

print(word_list, len(word_list))

['first', 'henry', 'fifth', 'sixth', 'third', 'merry', 'wives', 'night', 'dream', 'about', 'romeo', 'shrew', 'timon', 'titus', 'noble', 'lover', 'venus', 'might', 'never', 'riper', 'thine', 'light', 'flame', 'where', 'sweet', 'cruel', 'world', 'fresh', 'gaudy', 'churl', 'waste', 'grave', 'forty', 'shall', 'field', 'youth', 'proud', 'gazed', 'small', 'worth', 'being', 'asked', 'lusty', 'shame', 'child', 'count', 'blood', 'glass', 'whose', 'calls', 'april', 'prime', 'shalt', 'image', 'spend', 'gives', 'frank', 'lends', 'those', 'abuse', 'given', 'great', 'canst', 'alone', 'audit', 'leave', 'which', 'lives', 'hours', 'frame', 'every', 'dwell', 'excel', 'leads', 'there', 'frost', 'quite', 'walls', 'leese', 'their', 'still', 'place', 'usury', 'breed', 'times', 'could', 'death', 'worms', 'lifts', 'under', 'sight', 'looks', 'steep', 'adore', 'pitch', 'weary', 'tract', 'going', 'diest', 'music', 'sadly', 'annoy', 'tuned', 'chide', 'parts', 'happy', 'sings', 'prove', 'widow', 'shape', 'bosom', 

The next step is to find the frequnecy of each letter in our list of playable words. This list differs slightly from other charts of the frequency of letters in the English language at large (that I've encountered).

In [356]:
word_str = (str(word_list))
letter_freq = Counter(word_str) 
del letter_freq[' ']
del letter_freq[',']
del letter_freq["'"]
del letter_freq[']']
del letter_freq['[']
print (letter_freq)

Counter({'e': 2046, 's': 1690, 'a': 1380, 'r': 1208, 'o': 999, 'l': 979, 't': 970, 'i': 922, 'n': 836, 'd': 676, 'u': 599, 'c': 578, 'h': 516, 'p': 501, 'm': 423, 'b': 385, 'g': 381, 'y': 378, 'w': 350, 'f': 325, 'k': 302, 'v': 209, 'j': 46, 'x': 44, 'z': 40, 'q': 32})


Next, I want to make a simple equation to score a word based on its letter constituents and their frequency in our playable words.


Simple word scoring approach:

$ score(word) = \sum_{letter  \in  word}\frac{1}{rank(letter)} $

In [357]:
letters_by_frequency = "esaroltinduchpmbgywfkvjxzq"
letters_rank = {letter: 1/(i+1) for i, letter in enumerate(letters_by_frequency)}
    
def score_word(word):
    score = 0
    for letter in set(word):
        score += letters_rank[letter]
    return score

In [358]:
print("romeo:", score_word("romeo"))
print("henry:", score_word("henry"))

romeo: 1.5166666666666666
henry: 1.4935897435897436


In [359]:
def top_starting_word(word_list, topn=20):
    word_scores = []
    for word in tqdm(word_list):
        word_scores.append((word, score_word(word)))
    word_scores = sorted(word_scores, key=lambda x: x[1], reverse =True)
    for i in range(topn):
        print(f"{word_scores[i][0]} : {word_scores[i][1]}")

In [360]:
top_starting_word(word_list)

  0%|          | 0/3363 [00:00<?, ?it/s]

arose : 2.283333333333333
earls : 2.25
tears : 2.2261904761904763
rates : 2.2261904761904763
stare : 2.2261904761904763
arise : 2.208333333333333
raise : 2.208333333333333
aries : 2.208333333333333
aloes : 2.2
earns : 2.1944444444444446
snare : 2.1944444444444446
rased : 2.1833333333333336
dares : 2.1833333333333336
reads : 2.1833333333333336
dears : 2.1833333333333336
oates : 2.1761904761904765
cares : 2.1666666666666665
acres : 2.1666666666666665
scare : 2.1666666666666665
share : 2.16025641025641


Easy-peasy and without doing any real math we found that the best word to start with for five-letter words in the collected works of Sir William Shakespeare is AROSE, and it didn't take too long. Now lets see if there is a better approach and a way to compare the methods.

After some web-based research, I found a few things that could help improve our methodology.

Firstly, Wordle actually uses two lists of words, the first is the list that they pull the answers from (the word_list we just made is our analog here) and the second is a list of allowable words, i.e., words that can be played in the game but aren't necessarily on the list of answers. I found a version of this list online, which allows us a broader pool of words to start off with.

In [361]:
with open('wordle-allowed-guesses.txt', 'r') as file:
    words_allowed_guesses = file.read().splitlines()
    words_allowed_guesses = words_allowed_guesses + word_list
print (len(words_allowed_guesses))    

47546


Now that we've expanded our list of possible words to start with, let's think about the problem a little more mathematically.

Several bloggers that I came across working on this topic use the concept of entropy:
https://aditya-sengupta.github.io/coding/2022/01/13/wordle.html 

The main idea here is that we can characterize the average level of uncertainty inherent to a variable's possible outcomes.

More on that here:
https://en.wikipedia.org/wiki/Entropy_(information_theory)

For our example, we can implement this concept using this derived equation:

$ scoreImproved(word) = \sum_{Pattern  \in  Possible-Pattern} P(Pattern) * N_b WordsMatching(Pattern)$

In order to evaluate this equation we will need a function that can compare our guess word and the answer word. We have three possible outcomes for each letter in a word: 0 -  that lettter is not in the answer word, 1 - that letter is in the answer word, but not in that location, and 2 - that letter appears in the answer word in the current location.

In [362]:
def compare_words(guess_word, answer_word):
    res = []
    pos_not_found = []
    letters_not_found = []
    for (i, letter1), letter2 in zip(enumerate(guess_word), answer_word):
        if letter1 == letter2:
            res.append('2')
        else:
            res.append('0')
            pos_not_found.append(i)
            letters_not_found.append(letter2)
    for i in pos_not_found:
        if guess_word[i] in letters_not_found:
            res[i] = "1"
            letters_not_found.remove(guess_word[i])
    return "".join(res)

In [363]:
compare_words("romeo", "shrew")

'10020'

In [364]:
def score_word_improved(guess_word, possible_words):
    nb_words = len(possible_words)
    comparison_dict = defaultdict(int)
    for word in possible_words:
        comparison_dict[compare_words(guess_word, word)] += 1
    return sum(word_count / nb_words * word_count for word_count in comparison_dict.values())

In [365]:
def top_starting_word_improved(possible_guesses, possible_answers, topn=20):
    word_scores = []
    for word in tqdm(possible_guesses):
        word_scores.append((word, score_word_improved(word, possible_answers)))
    word_scores = sorted(word_scores, key=lambda x: x[1])
    for i in range(topn):
        print(f"{word_scores[i][0]} : {word_scores[i][1]}")

In [366]:
top_starting_word_improved(words_allowed_guesses, word_list)

  0%|          | 0/47546 [00:00<?, ?it/s]

aries : 66.49271483794237
rates : 71.24145108534044
tears : 72.8792744573297
laies : 73.71840618495395
saile : 74.18763009217959
tales : 74.628902765388
stare : 74.71989295272078
snare : 75.03925066904549
raise : 75.59351769253641
lanes : 77.05828129646152
earls : 78.75141242937855
aloes : 79.35504014272965
oates : 80.83050847457629
cares : 80.97026464466238
dares : 81.17662801070469
tires : 81.19565863812063
tries : 81.2051739518287
stale : 81.45911388641096
arise : 81.47279214986618
leans : 81.78382396669636


By using entropy, we have a new list of top starting words (and our previous top word didn't even make the list!)

Now, lets find a way to compare these two top words. We need a function that can filter out words that we have eliminated from our guess word.

I will be borrowing extensively from Francois Le Roix: https://medium.com/codex/solving-wordle-with-python-a729882e2e8a, who went through the trouble of building an evaluator for me.

In [367]:
def filter_words(possible_words, word, comparison_string):
    new_possible_words = []
    for w in possible_words:
        if compare_words(word, w) == comparison_string:
            new_possible_words.append(w)
    return new_possible_words

In [368]:
print(filter_words(word_list, "romeo", "10002"))

['cargo', 'varro', 'virgo', 'curio']


We now need a solver that uses these scoring methods (in this case we have two versions) to rank the words remaining after we've filtered our rule-out words and plays the best one.

In [369]:
def solver(comparison_string, word, possible_words):
    possible_words = filter_words(possible_words, word, comparison_string)
    if len(possible_words) == 0:
        print("No possible word in vocabulary.")
        return "", []
    else:
        words_sorted = sorted(possible_words, key=score_word, reverse=True)
        word = words_sorted[0]
        return word, possible_words

In [370]:
def solver_improved(comparison_string, word, possible_words):
    possible_words = filter_words(possible_words, word, comparison_string)
    if len(possible_words) == 0:
        print("No possible word in vocabulary.")
        return "", []
    else:
        words_sorted = sorted(possible_words, key=lambda x: score_word_improved(x, possible_words))
        word = words_sorted[0]
        return word, possible_words

Next, we went to simulate playing this method for every single word in our word_list and evalutate how the methodology works. We will be using two metrics: the mean number of attempts it takes to succesfully guess the correct word AND count (as well as list) the words that we were unable to solve.

In [371]:
def test_solution(answer_word, solver, starting_word="romeo"):
    word = starting_word
    possible_words = word_list
    attempts = 1
    while attempts < 20:
        if word == answer_word:
            return attempts
        attempts += 1
        comparison_string = compare_words(word, answer_word)
        word, possible_words = solver(comparison_string, word, possible_words)
    return attempts

def evaluate_solver(word_list, solver, starting_word="romeo"):
    solves = []
    for word in tqdm(word_list):
        solves.append((word, test_solution(word, solver, starting_word=starting_word)))
    mean_attempts = sum([solve[1] for solve in solves])/len(solves)
    print(f"Mean number of attempts: {mean_attempts}")
    failed = [solve[0] for solve in solves if solve[1] > 6]
    print(f"Failed words (more than 6 attempts): {len(failed)}")
    print(", ".join(failed))

In [372]:
evaluate_solver(word_list, solver, starting_word="arose")

  0%|          | 0/3363 [00:00<?, ?it/s]

Mean number of attempts: 4.096342551293488
Failed words (more than 6 attempts): 141
wives, riper, walls, yours, grows, fight, bears, deeds, taste, years, sinks, wards, watch, waves, fears, gates, kinds, catch, eager, falls, bases, hooks, sides, wears, fated, seeds, babes, fails, saves, eaves, taunt, hatch, sends, shave, river, sails, sayst, jaded, jests, feats, vales, rarer, stuff, caves, bills, mates, woods, wages, fells, marts, fates, meals, foxes, waved, waxed, lulls, kites, sours, fifes, fines, folds, wales, yokes, fanes, fares, faded, waxes, raves, cases, rawer, joyed, razes, banns, fazed, gower, sware, dated, bates, fills, wafer, james, sacks, gazes, gazer, games, freed, sells, wells, rears, yoked, vines, eater, jakes, farms, sizes, wakes, forks, hight, wares, ferry, vater, mazes, leavy, vails, farer, boxes, bakes, slays, gills, jaunt, kates, fives, girth, kated, jills, waded, lakes, gales, oozes, zeals, vides, gated, eases, feare, feard, warme, marke, beare, kinde, joyes, bowes,

In [373]:
evaluate_solver(word_list, solver, starting_word="aries")

  0%|          | 0/3363 [00:00<?, ?it/s]

Mean number of attempts: 4.057983942908118
Failed words (more than 6 attempts): 156
wives, riper, gazed, sings, yours, grows, faces, bears, deeds, taste, years, found, sinks, wards, waves, feeds, fears, gates, seals, finds, seems, kinds, catch, eager, falls, fools, bases, jacks, sides, wears, fated, seeds, seeks, bated, spark, babes, fails, saves, tents, eaves, hatch, barks, sends, shave, river, sails, jaded, jests, tight, gests, vales, rarer, flows, stuff, bills, mates, woods, wages, fells, fates, dined, waked, foxes, waxed, seats, lulls, kites, sours, fifes, fines, folds, wales, fanes, fares, gaged, faded, waxes, galls, warms, jowls, cases, rawer, joyed, razes, gower, sware, dated, bates, fills, wafer, james, sacks, gazes, gazer, poole, sells, wells, rears, yoked, vines, eater, jakes, farms, sizes, wakes, forks, dandy, hight, hests, wares, sakes, ferry, vater, seese, mazes, wooes, leavy, vails, farer, boxes, gapes, bakes, slays, gills, jaunt, kates, fives, kated, jills, waded, lakes,

In [374]:
evaluate_solver(word_list, solver_improved, starting_word="arose")

  0%|          | 0/3363 [00:00<?, ?it/s]

Mean number of attempts: 3.8682723758548914
Failed words (more than 6 attempts): 90
wears, seeds, jades, hears, eaves, hatch, binds, fates, napes, waxed, pates, wales, fanes, fares, galls, razes, lards, dated, bates, fills, dites, tower, pulls, cades, meeds, scare, hames, kibes, jakes, sizes, wakes, hight, mater, wares, sakes, cines, janus, cater, vater, pills, wanes, vails, fides, paced, wails, manes, bakes, gills, jaunt, daunt, kates, fives, kated, tames, jills, waded, gales, oozes, zeals, yells, gated, hacks, batch, mower, bayes, golds, eares, beare, waies, joyes, iades, saies, bowes, daies, yeare, sayes, wayes, vowes, binde, sowes, vaste, laies, weare, teare, rover, bents, tails, dears, lours, finis


In [375]:
evaluate_solver(word_list, solver_improved, starting_word="aries")

  0%|          | 0/3363 [00:00<?, ?it/s]

Mean number of attempts: 3.823074635741897
Failed words (more than 6 attempts): 83
wears, pears, jades, hears, eaves, hatch, jaded, pated, binds, cates, napes, waxed, kites, pates, doves, wales, fanes, galls, raves, razes, lards, fazed, gower, dated, bates, fills, wight, howls, layer, cades, meeds, scare, hames, jakes, wakes, hight, mater, wares, sakes, cines, dusty, janus, cater, vater, pills, wanes, vails, fides, wails, manes, bakes, gills, daunt, kates, fives, kated, jills, waded, gales, oozes, zeals, yells, baked, gated, batch, bayes, golds, eares, beare, joyes, boyes, yeare, sayes, wayes, binde, vaste, oates, weare, teare, fixes, tails, lames, lours


The performance of each of our methods for ranking the best word are remarkably close. In the simple version, AROSE was ever so slighlty worse in the number of guesses to get the correct the word but it was able to solve 15 more words than ARIES. In the improved version ARIES was slightly better at both metrics, but both words were able to solve more words than the previous method. In the end, it's not surprising that the word choice doesn't make a huge differece as they only differ by one letter. But it does seem that our improved scoring method is a better approach to the puzzle.

There are additional things we can do to increase our perfomance in the game, but because that is out of the scope of this excercise I will leave it here...for now.

